In [1]:
from Equili import *

# LOCATION OF PROBLEM FOLDER
folder_loc = '/home/elmanyer/Documents/BSC/MasterThesis/Code/execution/TS-'
case = 'UNSTRUCTURED'

# MESH INFORMATION:
ElementType = 0     # Element type -> 0: TRIANGLE  ; 1: QUADRILATERAL
ElementOrder = 1    # Element order -> 1: LINEAR  ; 2: QUADRATIC ; 3: CUBIC ...

# VACUUM VESSEL GEOMETRY (ITER):
epsilon = 0.32            # inverse aspect ratio
kappa = 1.7               # elongation
delta = 0.33              # triangularity
Rmax = 8                  # plasma major radius
Rmin = 4                  # plasma minor radius

##############################################################

# DECLARE OBJECT PROBLEM (folder with ALYA files)
directory = folder_loc + case
Problem = Equili(directory,ElementType,ElementOrder,Rmax,Rmin,epsilon,kappa,delta)

Problem.ReadMesh()
Problem.PlasmaEquilibrium()


COMPUTE INITIAL GUESS...Done!
INITIALISE LEVEL-SET...Done!
CLASSIFY ELEMENTS...Done!
APPROXIMATE INTERFACE...Done!
COMPUTE NUMERICAL INTEGRATION QUADRATURE AND SHAPE FUNCTIONS...Done!


In [2]:
plt.figure(figsize=(7,10))
plt.tricontourf(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=30, cmap='plasma')
plt.tricontour(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=[0], colors='k')
#plt.colorbar()

# PLOT NODES
plt.plot(Problem.X[:,0],Problem.X[:,1],'.',color='black')
Tmesh = Problem.T +1
# PLOT PLASMA REGION ELEMENTS
for e in Problem.PlasmaElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='red', linewidth=1)
# PLOT VACCUM ELEMENTS
for e in Problem.VacuumElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='black', linewidth=1)
# PLOT INTERFACE ELEMENTS
for e in Problem.PlasmaBoundElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='yellow', linewidth=1)
            
plt.show()

In [3]:
print(Problem.PlasmaBoundElems)

[ 935  936  938  939  941  942  943  944  945  946  948  968  971 1007
 1008 1010 1011 1012 1013 1014 1015 1017 1018 1020 1021 1022 1023 1029
 1031 1040 1042 1043 1045 1164 1167 1171 1173 1210 1212 1213 1219 1220
 1222 1223 1226 1229 1232 1238 1240 1243 1246 1384 1386 1391 1393 1563
 1565 1566 1568 1569 1571 1574 1577 1580 1582 1584 1586 1595 1597 1600
 1603 1859 1862 1863 1866 1867 1870 1871 1872 1875 1876 1879 1880 1882
 1883 1884 1895 2165 2166 2168 2248 2251 2255 2258 2261 2263 2264 2268
 2770 2773 2787 2788 2926 2927 2928 2929 2931 2934 2951 2952 2954 3011
 3014 3015 3016 3018 3021 3024 3242 3246 3249 3261 3367 3368 3369 3381
 3382 3383 3384 3385 3386 3389 3407 3431 3436 3438 3439 3444 3459 3461
 3465 3481 3497 3523 3524 3525 3536 3544 3548 3554 3555 3557 3558 3559
 3563 3567 3644 3672 3690 3692 3693 3694 3695 3696 3848 3851 3852 3854
 3855 3857 3858 3860 3861 3863 3865 3867 3871 3873 3876 4077 4384 4389
 4463 4481 4516]


In [4]:
element_index = 935

# READ NODAL COORDINATES 
Xe = Problem.X[Problem.T[element_index,:],:]
# PREPARE SMALL COMPUTATIONAL MESH
nmesh = 100
Xmesh, Ymesh = np.meshgrid(np.linspace(np.min(Xe[:,0])-0.1, np.max(Xe[:,0]), nmesh),
                           np.linspace(np.min(Xe[:,1])-0.1, np.max(Xe[:,1]), nmesh), indexing='ij')
xmesh = np.reshape(Xmesh, (nmesh*nmesh,1))
ymesh = np.reshape(Ymesh, (nmesh*nmesh,1))
zmesh = np.concatenate((xmesh, ymesh), axis=1)
# READ LEVEL-SET NODAL VALUES
LSe = Problem.PlasmaBoundLevSet[Problem.T[element_index,:]]  
# LOOK FOR THE NODE WHICH HAS DIFFERENT SIGN
pospos = np.where(LSe > 0)[0]
posneg = np.where(LSe < 0)[0]
if len(pospos) > len(posneg):  # 2 nodal level-set values are positive (outside plasma region)
    z0 = Xe[posneg,:][0]
    z1 = Xe[pospos,:]
else: # 2 nodal level-set values are negative (inside plasma region)
    z0 = Xe[pospos,:][0]
    z1 = Xe[posneg,:]

def N(Xe,z):
    npoints = np.shape(z)[0]
    J = np.concatenate((np.ones([3,1]),Xe), axis=1)
    denom = np.linalg.det(J)
    N = np.zeros([npoints, 3])
    for i in range(3):
        j = int((i+1)%3)
        k = int((i+2)%3)
        for n in range(npoints):
            N[n,i] = (Xe[j,0]*Xe[k,1]-Xe[k,0]*Xe[j,1]+(Xe[j,1]-Xe[k,1])*z[n,0]+(Xe[k,0]-Xe[j,0])*z[n,1])/denom
    return N

N = N(Xe,zmesh)

N1 = np.reshape(N[:,0], (nmesh,nmesh))
N2 = np.reshape(N[:,1], (nmesh,nmesh))
N3 = np.reshape(N[:,2], (nmesh,nmesh))

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(Xmesh, Ymesh, N1)
ax.plot_surface(Xmesh, Ymesh, N2)
ax.plot_surface(Xmesh, Ymesh, N3)
ax.plot_surface(Xmesh,Ymesh,np.zeros([nmesh,nmesh]), color='k')

plt.show()


In [11]:
element_index = 946

# READ NODAL COORDINATES 
Xe = Problem.X[Problem.T[element_index,:],:]

# PREPARE SMALL COMPUTATIONAL MESH
nmesh = 100
Xmesh, Ymesh = np.meshgrid(np.linspace(np.min(Xe[:,0])-0.1, np.max(Xe[:,0]), nmesh),
                           np.linspace(np.min(Xe[:,1])-0.1, np.max(Xe[:,1]), nmesh), indexing='ij')
xmesh = np.reshape(Xmesh, (nmesh*nmesh,1))
ymesh = np.reshape(Ymesh, (nmesh*nmesh,1))
zmesh = np.concatenate((xmesh, ymesh), axis=1)

# READ LEVEL-SET NODAL VALUES
LSe = Problem.PlasmaBoundLevSet[Problem.T[element_index,:]]  
# LOOK FOR THE NODE WHICH HAS DIFFERENT SIGN...
pospos = np.where(LSe > 0)[0]
posneg = np.where(LSe < 0)[0]
# ... PIVOT COORDINATES MATRIX ACCORDINGLY
if len(pospos) > len(posneg):  # 2 nodal level-set values are positive (outside plasma region)
    pos = np.concatenate((posneg,pospos),axis=0)
    Xe = Xe[pos]
    LSe = LSe[pos]
else: # 2 nodal level-set values are negative (inside plasma region)
    pos = np.concatenate((pospos,posneg),axis=0)
    Xe = Xe[pos]
    LSe = LSe[pos]

# NOW, THE FIRST ROW IN Xe AND FIRST ELEMENT IN LSe CORRESPONDS TO THE NODE ALONE IN ITS RESPECTIVE REGION (INSIDE OR OUTSIDE PLASMA REGION)

def z(r,Xe,edge):
    # Computes restrictions associated to the element edge
    z = ((Xe[edge,1]-Xe[0,1])*r+Xe[0,1]*Xe[edge,0]-Xe[edge,1]*Xe[0,0])/(Xe[edge,0]-Xe[0,0])
    return z

def fun(r,Xe,LSe,edge):
    
    def N0(r,z,Xe):
        j = 1
        k = 2
        N = Xe[j,0]*Xe[k,1]-Xe[k,0]*Xe[j,1]+(Xe[j,1]-Xe[k,1])*r+(Xe[k,0]-Xe[j,0])*z
        return N
    def Nedge(r,z,Xe,edge):
        j = (edge+1)%3
        k = (edge+2)%3
        N = Xe[j,0]*Xe[k,1]-Xe[k,0]*Xe[j,1]+(Xe[j,1]-Xe[k,1])*r+(Xe[k,0]-Xe[j,0])*z
        return N
    
    f = N0(r,z(r,Xe,edge),Xe)*LSe[0] + Nedge(r,z(r,Xe,edge),Xe,edge)*LSe[edge]
    return f

from scipy import optimize

edge1 = 1
sol1 = optimize.root(fun, Xe[0,0], args=(Xe,LSe,edge1))
edge2 = 2
sol2 = optimize.root(fun, Xe[0,0], args=(Xe,LSe,edge2))

print(sol1.x, z(sol1.x,Xe,edge1))
print(sol2.x, z(sol2.x,Xe,edge2))


plt.figure(figsize=(7,10))
plt.tricontourf(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=30, cmap='plasma')
plt.tricontour(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=[0], colors='k')
for i in range(Problem.n):
    plt.plot([Problem.X[int(Tmesh[element_index,i])-1,0], Problem.X[int(Tmesh[element_index,int((i+1)%Problem.n)])-1,0]], 
            [Problem.X[int(Tmesh[element_index,i])-1,1], Problem.X[int(Tmesh[element_index,int((i+1)%Problem.n)])-1,1]], color='yellow', linewidth=1)

plt.plot(sol1.x, z(sol1.x,Xe,edge1), '.', color='red',markersize=10)
plt.plot(sol2.x, z(sol2.x,Xe,edge2), '.', color='red',markersize=10)
plt.plot((sol1.x,sol2.x),(z(sol1.x,Xe,edge1),z(sol2.x,Xe,edge2)),color='red')

plt.xlim(np.min(Xe[:,0])-0.05,np.max(Xe[:,0])+0.05)
plt.ylim(np.min(Xe[:,1])-0.05,np.max(Xe[:,1])+0.05)




[7.69373619] [-1.37354223]
[7.69760224] [-1.362693]


(-1.585898, -1.3126929999999999)

In [6]:

def InterfaceLinearApproximation(Problem, element_index):
    # READ NODAL COORDINATES 
    Xe = Problem.X[Problem.T[element_index,:],:]

    # READ LEVEL-SET NODAL VALUES
    LSe = Problem.PlasmaBoundLevSet[Problem.T[element_index,:]]  
    # LOOK FOR THE NODE WHICH HAS DIFFERENT SIGN...
    pospos = np.where(LSe > 0)[0]
    posneg = np.where(LSe < 0)[0]
    # ... PIVOT COORDINATES MATRIX ACCORDINGLY
    if len(pospos) > len(posneg):  # 2 nodal level-set values are positive (outside plasma region)
        pos = np.concatenate((posneg,pospos),axis=0)
        Xe = Xe[pos]
        LSe = LSe[pos]
    else: # 2 nodal level-set values are negative (inside plasma region)
        pos = np.concatenate((pospos,posneg),axis=0)
        Xe = Xe[pos]
        LSe = LSe[pos]

    # NOW, THE FIRST ROW IN Xe AND FIRST ELEMENT IN LSe CORRESPONDS TO THE NODE ALONE IN ITS RESPECTIVE REGION (INSIDE OR OUTSIDE PLASMA REGION)

    def z(r,Xe,edge):
        # Computes restrictions associated to the element edge
        z = ((Xe[edge,1]-Xe[0,1])*r+Xe[0,1]*Xe[edge,0]-Xe[edge,1]*Xe[0,0])/(Xe[edge,0]-Xe[0,0])
        return z

    def fun(r,Xe,LSe,edge):
        
        def N0(r,z,Xe):
            j = 1
            k = 2
            N = Xe[j,0]*Xe[k,1]-Xe[k,0]*Xe[j,1]+(Xe[j,1]-Xe[k,1])*r+(Xe[k,0]-Xe[j,0])*z
            return N
        def Nedge(r,z,Xe,edge):
            j = (edge+1)%3
            k = (edge+2)%3
            N = Xe[j,0]*Xe[k,1]-Xe[k,0]*Xe[j,1]+(Xe[j,1]-Xe[k,1])*r+(Xe[k,0]-Xe[j,0])*z
            return N
        
        f = N0(r,z(r,Xe,edge),Xe)*LSe[0] + Nedge(r,z(r,Xe,edge),Xe,edge)*LSe[edge]
        return f

    from scipy import optimize

    interface = np.zeros([2,2])
    for i, edge in enumerate([1,2]):
        sol = optimize.root(fun, Xe[0,0], args=(Xe,LSe,edge))
        interface[i,:] = [sol.x, z(sol.x,Xe,edge)]
    
    return interface


Ninterface = len(Problem.PlasmaBoundElems)
InterfaceCoordinates = np.zeros([2*Ninterface,2])

for i, element in enumerate(Problem.PlasmaBoundElems):
    interface = InterfaceLinearApproximation(Problem, element)
    InterfaceCoordinates[2*i:2*(i+1),:] = interface


plt.figure(figsize=(7,10))
plt.tricontourf(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=30, cmap='plasma')
plt.tricontour(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=[0], colors='k')
#plt.colorbar()

# PLOT NODES
plt.plot(Problem.X[:,0],Problem.X[:,1],'.',color='black')
Tmesh = Problem.T +1
# PLOT PLASMA REGION ELEMENTS
for e in Problem.PlasmaElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='red', linewidth=1)
# PLOT VACCUM ELEMENTS
for e in Problem.VacuumElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='black', linewidth=1)
# PLOT INTERFACE ELEMENTS
for e in Problem.PlasmaBoundElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='yellow', linewidth=1)
# PLOT INTERAFACE LINEAR APROXIMATION
for i in range(Ninterface):
    plt.plot(InterfaceCoordinates[2*i:2*(i+1),0],InterfaceCoordinates[2*i:2*(i+1),1],color='green', linewidth=3)
            
plt.show()


/tmp/ipykernel_11315/1831074072.py:48: DeprecationWarning: setting an array element with a sequence. This was supported in some cases where the elements are arrays with a single element. For example `np.array([1, np.array([2])], dtype=int)`. In the future this will raise the same ValueError as `np.array([1, [2]], dtype=int)`.
  interface[i,:] = [sol.x, z(sol.x,Xe,edge)]


In [7]:
plt.figure(figsize=(7,10))
plt.tricontourf(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=30, cmap='plasma')
plt.tricontour(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=[0], colors='k')
#plt.colorbar()

# PLOT NODES
plt.plot(Problem.X[:,0],Problem.X[:,1],'.',color='black')
Tmesh = Problem.T +1
# PLOT PLASMA REGION ELEMENTS
for e in Problem.PlasmaElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='red', linewidth=1)
# PLOT VACCUM ELEMENTS
for e in Problem.VacuumElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='black', linewidth=1)
# PLOT INTERFACE ELEMENTS
for e in Problem.PlasmaBoundElems:
    for i in range(Problem.n):
        plt.plot([Problem.X[int(Tmesh[e,i])-1,0], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,0]], 
                [Problem.X[int(Tmesh[e,i])-1,1], Problem.X[int(Tmesh[e,int((i+1)%Problem.n)])-1,1]], color='yellow', linewidth=1)
# PLOT INTERAFACE LINEAR APROXIMATION
for i in range(Ninterface):
    plt.plot(InterfaceCoordinates[2*i:2*(i+1),0],InterfaceCoordinates[2*i:2*(i+1),1],color='green', linewidth=3)
            
plt.xlim(6.5,7.5)
plt.ylim(2,3)            

plt.show()




In [8]:
interface = 3

element_index = Problem.PlasmaBoundElems[interface]  # index of element in which is contained the interface
Xe = Problem.X[Problem.Tinter[interface,:],:]   # nodal coordinates of element, where the first row is the node common to both edges intersecting with the interface
Xeint = Problem.InterfaceCoordinates[2*interface:2*(interface+1),:]   # coordinates of intersections between interface and edges

# MODIFIED NODAL MATRIX AND CONECTIVITIES, ACCOUNTING FOR 3 SUBTRIANGLES 
Xemod = np.concatenate((Xe, Xeint), axis=0)
Temod = np.zeros([3, Problem.n], dtype = int)  # connectivities for 3 subtriangles

Temod[0,:] = [0, 3, 4]  # first triangular subdomain is common node and intersection nodes

# COMPARE DISTANCE INTERFACE-(EDGE NODE)
edge = 1
distance1 = np.linalg.norm(Xeint[edge-1,:]-Xe[edge,:])
edge = 2
distance2 = np.linalg.norm(Xeint[edge-1,:]-Xe[edge,:])

if distance1 < distance2:
    Temod[1,:] = [3, 1, 2]
    Temod[2,:] = [3, 4, 2]
if distance1 > distance2:
    Temod[1,:] = [4, 2, 1]
    Temod[2,:] = [4, 3, 1]
    

# PREPARE SMALL COMPUTATIONAL MESH
nmesh = 100
Xmesh, Ymesh = np.meshgrid(np.linspace(np.min(Xe[:,0])-0.1, np.max(Xe[:,0]), nmesh),
                           np.linspace(np.min(Xe[:,1])-0.1, np.max(Xe[:,1]), nmesh), indexing='ij')
xmesh = np.reshape(Xmesh, (nmesh*nmesh,1))
ymesh = np.reshape(Ymesh, (nmesh*nmesh,1))
zmesh = np.concatenate((xmesh, ymesh), axis=1)

plt.figure(figsize=(7,10))
plt.tricontourf(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=30, cmap='plasma')
plt.tricontour(Problem.X[:,0],Problem.X[:,1], Problem.PHI_inner0, levels=[0], colors='k')
for i in range(Problem.n):
    plt.plot([Problem.X[int(Tmesh[element_index,i])-1,0], Problem.X[int(Tmesh[element_index,int((i+1)%Problem.n)])-1,0]], 
            [Problem.X[int(Tmesh[element_index,i])-1,1], Problem.X[int(Tmesh[element_index,int((i+1)%Problem.n)])-1,1]], color='yellow', linewidth=3)

"""
# PLOT INTERFACE LINEAR APPROXIMATION
plt.plot(Xeint[:,0], Xeint[:,1], '.', color='red',markersize=10)
plt.plot(Xeint[:,0], Xeint[:,1],color='red', linewidth=3)
"""

"""
# PLOT SEGMENTS INTERFACE-(EDGE NODE)
for edge in [1,2]:
    plt.plot([Xeint[edge-1,0], Xe[edge,0]], [Xeint[edge-1,1], Xe[edge,1]], '.', color='green',markersize=10)
    plt.plot([Xeint[edge-1,0], Xe[edge,0]], [Xeint[edge-1,1], Xe[edge,1]], color='green', linewidth=3)
    """
    
# PLOT TRIANGULAR SUBDOMAINS
colorlist = ['pink', 'cyan', 'olive']
markerlist = ['^', '<', '>']
for i in range(3):  # for each subdomain
    for j in range(Problem.n):   # plot edges
        plt.plot([Xemod[Temod[i,j],0], Xemod[Temod[i,int((j+1)%Problem.n)],0]], [Xemod[Temod[i,j],1], Xemod[Temod[i,int((j+1)%Problem.n)],1]],
                 linestyle = '-', color = colorlist[i], linewidth=2 , marker=markerlist[i])

plt.xlim(np.min(Xe[:,0])-0.05,np.max(Xe[:,0])+0.05)
plt.ylim(np.min(Xe[:,1])-0.05,np.max(Xe[:,1])+0.05)


(-1.759103, -1.485898)

In [10]:
# INVERSE MAPPING TO FIND INTERFACE COORDINATES IN REFERENCE ELEMENT
from ShapeFunctions import *

interface = 3
element_index = Problem.PlasmaBoundElems[interface]

# LOOK FOR INTERFACE POINTS IN REFERENCE ELEMENT
Xg = np.zeros([2,2])
for j in range(2):
    X = Problem.InterfaceCoordinates[2*interface+j,:]
    print(X)
    # READ ELEMENTAL NODAL COORDINATES 
    Xe = Problem.X[Problem.T[element_index,:],:]
    # DEFINE THE NONLINEAR SYSTEM 
    def fun(Xg, X, Xe):
        f = np.array([-X[0],-X[1]])
        for i in range(Problem.n):
            Nig, foo, foo = ShapeFunctionsReference(Xg,Problem.ElType, Problem.ElOrder, i+1)
            f[0] += Nig*Xe[i,0]
            f[1] += Nig*Xe[i,1]
        return f
    # SOLVE USING NONLINEAR SOLVER
    XG0 = np.array([1/2,1/2])  # INITIAL GUESS FOR ROOT SOLVER
    sol = optimize.root(fun, XG0, args=(X,Xe))
    print(sol)
    Xg[j,:] = sol.x
    
fig, axs = plt.subplots(1, 2, figsize=(14,8))
# PLOT IN PHYSICAL SPACE
for i in range(Problem.n):
    axs[0].plot([Problem.X[int(Tmesh[element_index,i])-1,0], Problem.X[int(Tmesh[element_index,int((i+1)%Problem.n)])-1,0]], 
            [Problem.X[int(Tmesh[element_index,i])-1,1], Problem.X[int(Tmesh[element_index,int((i+1)%Problem.n)])-1,1]], color='black', linewidth=3)
# PLOT INTERFACE LINEAR APPROXIMATION
Xeint = Problem.InterfaceCoordinates[2*interface:2*(interface+1),:]   # coordinates of intersections between interface and edges
axs[0].plot(Xeint[:,0], Xeint[:,1], '.', color='red',markersize=10)
axs[0].plot(Xeint[:,0], Xeint[:,1],color='red', linewidth=3)

# PLOT IN REFERENCE SPACE
Xref = np.array([[0,0], [1,0], [0,1]])
Tref = np.array([0,1,2])
for i in range(Problem.n):
    axs[1].plot([Xref[Tref[i],0], Xref[Tref[(i+1)%Problem.n],0]], 
            [Xref[Tref[i],1], Xref[Tref[(i+1)%Problem.n],1]], color='black', linewidth=3)
# PLOT INTERFACE LINEAR APPROXIMATION
axs[1].plot(Xg[:,0], Xg[:,1], '.', color='red',markersize=10)
axs[1].plot(Xg[:,0], Xg[:,1],color='red', linewidth=3)


[ 7.55856644 -1.709103  ]
    fjac: array([[ 0.85593875,  0.51707722],
       [-0.51707722,  0.85593875]])
     fun: array([-1.77635684e-16,  1.57705848e-16])
 message: 'The solution converged.'
    nfev: 7
     qtf: array([-7.04991633e-17,  2.26837913e-16])
       r: array([-0.27467257,  0.30485803, -0.49955879])
  status: 1
 success: True
       x: array([0.70716782, 0.29283218])
[ 7.61965627 -1.56994364]
    fjac: array([[-0.71028675, -0.70391244],
       [-0.70391244,  0.71028675]])
     fun: array([8.8817842e-16, 0.0000000e+00])
 message: 'The solution converged.'
    nfev: 12
     qtf: array([-6.30861366e-16, -6.25199841e-16])
       r: array([ 0.28157618, -0.03773378, -0.24605038])
  status: 1
 success: True
       x: array([-8.54005623e-16,  1.96562682e-01])
